## Introdução
Este notebook extrai o texto do teor das decisões jurídicas capturadas no e-Saj com o webscrapper. Do diretório raiz do projeto é possível acessar o diretório _dados_. Dentro dele, o diretório _raw_data_ contém arquivos csv com código do processo, classe, assunto, magistrado, comarca, foro, vara, data da disponibilização no e-Saj e teor de 500 decisões de segunda instância. Uma vez que existem muitos arquivos csv, é preciso percorrê-los, um a um, aplicando o tratamento textual.

As informações estruturadas supracitadas se repetem no teor dos documentos, então é necessário remover este conteúdo, o que é feito com uma expressão regular. Em seguida, os documentos de texto são salvos em formato txt para posterior treinamento de modelos de linguagem.

In [1]:
%load_ext autoreload
%autoreload 2

import string
import unidecode
import re
import nltk
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import sys
from tqdm import tqdm
import preprocessing

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


#### Gera um txt para cada teor

In [2]:
caminho = "dados/raw_data/"
expr_teor = re.compile('(\\n\\t\\t\\t\\t\\t\\t\\t\\t\\t\\t\\t\\t\\t\\n\\n\\n.*TRIBUNAL DE JUSTI.*V[iI][sS][tT][oO][sS]|\\n\\t\\t\\t\\t\\t\\t\\t\\t\\t\\t\\t\\t\\t\\n\\n\\nTRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\\n\\n.*Juiz.* de Direito:.+?\\n)', re.S)
tokens = 0
documentos = 0

for arquivo in tqdm(os.listdir(caminho)):    
    dados = pd.read_csv(caminho + arquivo, usecols=['id','Teor'])
    lista = dados['Teor'].apply(lambda x: True if expr_teor.search(str(x)) else False)
    dados = dados[lista]
    textos_raw = dados['Teor'].apply(lambda x: re.sub('[\n\r\t]+', ' ', re.sub(expr_teor, '', str(x))))
    for i, dado in enumerate(textos_raw.values):
        documento = open("dados/corpus_tratado/" + dados['id'][i]+ ".txt", mode="w+",encoding="utf-8")
        preprocessing.preprocessar_texto(textos_raw.values[i], documento, tokens)
        documento.close()

100%|██████████| 435/435 [3:17:03<00:00, 27.18s/it]  


### Gera um txt para o corpus inteiro

In [3]:
caminho_corpus = "dados/corpus_tratado/"
corpus_tratado = open('dados/corpus_tratado/single/corpus_tratado.txt', 'w+', encoding='utf8')
for documento in tqdm(glob.glob("dados/corpus_tratado/*.txt")):
    doc = open(documento, 'r', encoding='utf8')
    for frase in doc:
        corpus_tratado.write(frase)
corpus_tratado.close() 

100%|██████████| 40019/40019 [07:10<00:00, 92.97it/s] 


In [4]:
tokens

0